In [ ]:
import pandas as pd
import os
from glob import glob

In [ ]:
raiz = '../databases'

arquivos_csv = []
arquivos_csv.extend(glob(os.path.join(raiz, 'ouro_npk_*.csv')))
arquivos_csv.sort()

df_balanco = pd.DataFrame()

fontes = ('Carcaça Bovina',
          'Dejetos Galináceos',
          'Dejetos Suínos',
          'Deposição Atmosférica',
          'Fertilizante orgânico vinhaça',
          'Fertilizante Sintético',
          'Fixação Bio N',
          'Outras Produções Agrícolas',
          'Principais Produções Agrícolas',
          'Sementes'
          )

In [ ]:
arquivos_csv

In [ ]:
df_balanco = pd.DataFrame()

df = (
    pd.concat([
        pd.read_csv(arquivo)
          .assign(valor=lambda v: v.valor/1000,
                  fluxo=lambda x: 'Saída' if arquivo in (arquivos_csv[0], arquivos_csv[7], arquivos_csv[8]) else 'Entrada')
        for arquivo in arquivos_csv]
        #for arquivo, fonte in zip(arquivos_csv, fontes)]
    )
    .query('uf != 1')
    .sort_values(
        by=['fonte', 'ano', 'uf', 'elemento', 'fluxo'],
        key=lambda x: x.map({
            'Fertilizantes Sintéticos': 0,
            'Fertilizante orgânico vinhaça': 1,
            'Dejetos galináceos': 2,
            'Dejetos suínos': 3,
            'Fixação biológica de N': 4,
            'Sementes': 5,
            'Deposição Atmosférica': 6,
            'Principais produções agrícolas': 7,
            'Outras produções agrícolas': 8,
            'Carcaça Bovina': 9
        })
    )
    .set_index(['uf', 'ano', 'fluxo', 'fonte', 'elemento'])
    .squeeze()
    .rename("valor_mt")
)

df.to_frame().to_parquet(f'{raiz}/ouro_npk-balanco.parquet')

df

In [ ]:
# df_balanco = (df_balanco.groupby(['uf', 'ano'])
#     [['fonte', 'elemento', 'valor', 'fluxo']]
#     .apply(lambda x: x)
#     .sort_values(
#         by=['elemento', 'uf'],
#         key=lambda x: x.map({'N': 0, 'P': 1, 'K': 2})
#         )
#         .reset_index()
#         .drop('level_2', axis='columns')
#         .convert_dtypes()
#                 )

# df_balanco.to_csv(f'{raiz}/ouro_npk-balanco.csv', index=False)